In [1]:
import cv2
import tkinter as tk
from deepface import DeepFace
from PIL import Image, ImageTk

# Create the main GUI window
root = tk.Tk()
root.attributes('-fullscreen', True)  # Set the window geometry
root.title("Face Emotion Recognition")

headline_label = tk.Label(root, text="Face Emotion Recognition System", font=("Helvetica", 24, "bold"), pady=20)
headline_label.pack()

# Create a frame to hold the camera feed label with padding
outer_frame = tk.Frame(root, bg="black")
outer_frame.pack(pady=20, padx=(80, 80), expand=tk.YES, fill=tk.BOTH)  # Center and add padding

# Create a label to display video feed
video_label = tk.Label(outer_frame, bg="white")
video_label.pack(expand=tk.YES, fill=tk.BOTH)

# Create a label to display emotion quotes
quote_label = tk.Label(root, text="", font=("Helvetica", 20))
quote_label.pack()

# Emotion-quote dictionary
emotion_quotes = {
    "angry": "Stay calm and keep moving forward.",
    "happy": "Happiness is a choice. Choose it.",
    "sad": "Every cloud has a silver lining.",
    "neutral": "Sometimes, no emotion is the best emotion.",
    "surprise": "Life is full of surprises. Embrace them.",
    "fear": "Fear is temporary; regret is forever."
}

# Function to handle camera capture and GUI update
def update_frame():
    ret, frame = cap.read()
    resized_frame = cv2.resize(frame, (250, 250))
    result = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False)

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray, 1.1, 4)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    dominant_emotion = result[0]['dominant_emotion']
    quote = emotion_quotes.get(dominant_emotion.lower(), "No quote available.")

    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(frame, dominant_emotion, (50, 50), font, 2, (0, 0, 255), 2, cv2.LINE_4)

    # Convert OpenCV frame to Tkinter format
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(img)
    img = ImageTk.PhotoImage(image=img)

    video_label.imgtk = img
    video_label.config(image=img)

    quote_label.config(text=quote)  # Update emotion quote in the GUI

    root.update_idletasks()  # Update the GUI window without handling events

    if cap.isOpened():
        root.after(30, update_frame)  # Increase the delay for smoother output

# Function to start the camera capture
def start_capture():
    global cap
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        raise IOError("Cannot Open Webcam")

    update_frame()  # Start the frame update loop

# Function to stop the camera capture
def stop_capture():
    if cap.isOpened():
        cap.release()
    cv2.destroyAllWindows()
    root.destroy()  # Quit the Tkinter application

# Frame to hold buttons
button_frame = tk.Frame(root)
button_frame.pack(pady=20)

# Start button
start_button = tk.Button(button_frame, text="Start", command=start_capture, bg="green", fg="white", font=("Helvetica", 20))
start_button.pack(side=tk.LEFT, padx=10)

# Quit button
quit_button = tk.Button(button_frame, text="Quit", command=stop_capture, bg="red", fg="white", font=("Helvetica", 20))
quit_button.pack(side=tk.LEFT, padx=10)

# Create the cascade classifier outside the function
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Start the GUI event loop
root.mainloop()


pip install -U ipykernel
